In [110]:
import sys
sys.path.append("..")
import functools

import numpy as np
import torch
import scipy.linalg
import scipy.optimize
import scipy.integrate
import matplotlib.pyplot as plt
from dict_minimize.torch_api import minimize
import functorch
import pyequion2

import tmcdiff

In [111]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
def reynolds_number(flow_velocity, pipe_diameter, TK=298.15,
                    kinematic_viscosity = None): #Dimensionless
    """
        Calculates Reynolds number of water from velocity and diameter
    """
    kinematic_viscosity = kinematic_viscosity or pyequion2.water_properties.water_kinematic_viscosity(TK)
    return flow_velocity*pipe_diameter/kinematic_viscosity


def darcy_friction_factor(flow_velocity=float,
                          pipe_diameter=float,
                          TK=298.15,
                          kinematic_viscosity=None):
    reynolds = reynolds_number(flow_velocity, pipe_diameter, TK, kinematic_viscosity)
    if reynolds < 2300:
        return 64/reynolds
    else: #Blasius
        return 0.316*reynolds**(-1./4)
    

def get_shear_velocity(flow_velocity,
                   pipe_diameter,
                   TK=298.15,
                   kinematic_viscosity=None):
    f = darcy_friction_factor(flow_velocity, pipe_diameter, TK, kinematic_viscosity)
    return np.sqrt(f/8.0)*flow_velocity

In [78]:
eqsys = pyequion2.EquilibriumBackend(["C", "Ca", "Na", "Cl"], from_elements=True, backend="torch", logbase="e",
                                     activity_model="DEBYE")
eqsys_front = pyequion2.EquilibriumSystem(["C", "Ca", "Na", "Cl"], from_elements=True,
                                           activity_model="DEBYE")

In [99]:
TK = 298.15
bulk_velocity = 1.0
pipe_diameter = 1e-2
shear_velocity = get_shear_velocity(bulk_velocity, pipe_diameter, TK)
kinematic_viscosity = 1e-6
c_nahco3 = 0.020
c_cacl2 = 0.010
molal_balance = {'C':c_nahco3, 'Ca':c_cacl2, 'Na':c_nahco3, 'Cl':2*c_cacl2}
molal_balance = {k:v*1e0 for k, v in molal_balance.items()}

builder = tmcdiff.TransportBuilder(eqsys, TK, shear_velocity,
                           molal_balance, ["Calcite"],
                           kreaction="inf")
builder.make_grid(20, 5)
builder.set_species()

In [103]:
sol, _ = eqsys_front.solve_equilibrium_mixed_balance(TK, molal_balance=molal_balance)
sol.saturation_indexes

{'Aragonite': 1.5992366772106408,
 'Calcite': 1.743006524602876,
 'Halite': -5.161929565992914,
 'Vaterite': 1.1765750303885767}

In [104]:
builder.set_initial_guess_from_bulk()
solsimple = builder.solve_lma(simplified=True);
solfull = builder.solve_lma(simplified=False);

In [105]:
def f(t, x):
    builder.cbulk = {el:x[i] for i, el in enumerate(builder.eqsys.solute_elements)}
#     solsimple = builder.solve_lma(simplified=True);
    solfull = builder.solve_lma(simplified=False);
    res = builder.fluxes().detach().numpy()[:-1, -1]*4/pipe_diameter
    return res

In [106]:
x = np.array([molal_balance[el] for el in builder.eqsys.solute_elements])

In [107]:
solsimple = builder.solve_lma(simplified=True);
solfull = builder.solve_lma(simplified=False);
solsimple.success, solfull.success

(True, True)

In [115]:
builder.get_logc().numpy()

array([[ -5.454978 ,  -5.677202 ,  -5.9362617,  -6.2140293,  -6.4824686,
         -6.718816 ,  -6.913115 ,  -7.0662255,  -7.1844816,  -7.275424 ,
         -7.345734 ,  -7.400655 ,  -7.4440846,  -7.4788756,  -7.5070896,
         -7.530251 ,  -7.549464 ,  -7.565567 ,  -7.5791855,  -7.5908   ],
       [-12.434015 , -11.886845 , -11.3686905, -10.90308  , -10.509447 ,
        -10.193228 ,  -9.948345 ,  -9.762688 ,  -9.622888 ,  -9.517199 ,
         -9.436458 ,  -9.373926 ,  -9.324786 ,  -9.285613 ,  -9.253957 ,
         -9.228055 ,  -9.206612 ,  -9.188679 ,  -9.173532 ,  -9.160634 ],
       [ -5.6065874,  -5.295944 ,  -5.082362 ,  -4.948311 ,  -4.869468 ,
         -4.8233314,  -4.795494 ,  -4.7779555,  -4.766398 ,  -4.7584553,
         -4.7527905,  -4.748618 ,  -4.7454596,  -4.7430115,  -4.741076 ,
         -4.7395196,  -4.7382483,  -4.737197 ,  -4.736317 ,  -4.735574 ],
       [-12.09904  , -11.315721 , -10.646648 , -10.09232  ,  -9.649251 ,
         -9.305266 ,  -9.044126 ,  -8.848443 ,  

In [109]:
solver = scipy.integrate.ode(f)
solver.set_integrator("vode")
solver.set_initial_value(x)
xarr = [x.copy()]
tarr = [0.0]
tmax = 1e-3
while solver.successful():
    solver.integrate(tmax, step=True)
    xarr.append(solver.y)
    tarr.append(solver.t)
    if solver.t > tmax:
        break
    print(solver.t, solver.y)

3.432259064882502e-08 [0.01 0.02 0.02 0.02]
1.26642615058863e-06 [0.00999985 0.02       0.01999985 0.02      ]
1.358746174998668e-05 [0.00999844 0.02       0.01999844 0.02      ]
0.00013679781774396718 [0.00998429 0.02       0.01998429 0.02      ]
0.0002196752699975134 [0.00997479 0.02       0.01997479 0.02000001]
0.0003025527222510596 [0.0099653  0.02       0.0199653  0.02000001]
0.00034537366894605345 [0.00996041 0.02       0.01996041 0.02000001]
0.0003881946156410473 [0.00995552 0.02       0.01995551 0.02000001]
0.00043239664696690755 [0.00995048 0.02       0.01995047 0.02000001]
0.0004765986782927678 [0.00994544 0.02       0.01994544 0.02000001]
0.0005210388731171086 [0.00994038 0.02       0.01994038 0.02000001]
0.0005654790679414494 [0.00993533 0.02       0.01993533 0.02000001]
0.000610293585657545 [0.00993024 0.02       0.01993024 0.02000001]
0.0006551081033736405 [0.00992516 0.02       0.01992515 0.02000002]
0.0006988462063269213 [0.00992022 0.02       0.01992022 0.02000002]
0.0

In [ ]:
xarr = np.vstack(xarr)
tarr = np.array(tarr)

In [ ]:
for i, el in enumerate(eqsys.solute_elements):
    plt.plot(tarr, xarr[:, i], label=el)
plt.legend()